# Data preprocessing on the prepared diabetic dataset.
After interpretation of ICD code and flitering records with diabetes diagnosis, we further perform data preprocessing. It is important to note that there isn't much of a clean separation between steps, and there is a lot of back and forth interations when trying different approaches to data cleanning and feature engineering.

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Some options for Pandas and Seaborn:

In [2]:
# Always display all the columns
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 200) 

# Plain Seaborn figures with matplotlib color codes mapped to the default seaborn palette 
sns.set(style="white", color_codes=True)

Import the flitered file containing the health dataset as a Pandas dataframe

In [3]:
df = pd.read_csv("diabetes_data_preprocessed.csv")

## Quick data exploration
When looking at a dataset for the first time, we have basically no idea what to expect. Getting some basic parameters, like number of columns and rows, column headings, and the data format of the columns should be the first step.

In [4]:
df.shape

(38314, 50)

In [5]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'payer_code', 'medical_specialty', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted', 'c_diag1', 'c_diag2', 'c_diag3'], dtype='object')

In [6]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide           

We should now look at the data itself. With head() we can print the first few lines in the dataframe.

In [7]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,c_diag1,c_diag2,c_diag3
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,Diabetes,?,?
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30,Others,Diabetes,Diabetes
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,Others,Diabetes,Others
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO,Others,Diabetes,Circulatory
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO,Neoplasms,Neoplasms,Diabetes


Now we know what the dataframe roughly looks like. To get an idea what to expect across all the 40,000 rows, we can let Pandas summarize the columns containing numerical values:

In [8]:
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,3.831400e+04,3.831400e+04,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000,38314.000000
mean,1.507911e+08,4.974828e+07,2.077048,3.479929,5.617868,3.939369,41.509892,1.157227,14.603852,0.324659,0.219685,0.611604,6.431983
std,1.002594e+08,3.884212e+07,1.493341,5.303000,4.079970,2.786040,19.382049,1.581373,7.623154,1.162165,0.999072,1.350888,2.201603
min,1.668000e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,7.251941e+07,1.849495e+07,1.000000,1.000000,1.000000,2.000000,30.000000,0.000000,9.000000,0.000000,0.000000,0.000000,5.000000
50%,1.345140e+08,4.177708e+07,1.000000,1.000000,7.000000,3.000000,43.000000,1.000000,13.000000,0.000000,0.000000,0.000000,6.000000
75%,2.080023e+08,8.524751e+07,3.000000,3.000000,7.000000,5.000000,55.000000,2.000000,19.000000,0.000000,0.000000,1.000000,9.000000
max,4.438475e+08,1.894815e+08,8.000000,28.000000,20.000000,14.000000,118.000000,6.000000,68.000000,37.000000,54.000000,21.000000,16.000000


Based on this first look at the data, I identified a few columns containing interesting information. 
Let's start having a more detailed look at the data by breaking down the dataset based on primary diagnosis:

In [9]:
diseases = df["c_diag1"].value_counts()
diseases

Circulatory        8885
Diabetes           8869
Others             6174
Respiratory        4862
Digestive          3063
Musculoskeletal    2314
Injury             1759
Genitourinary      1433
Neoplasms           949
?                     6
Name: c_diag1, dtype: int64

In [10]:
encounter_number = df['encounter_id'].count()
patient_number = df['patient_nbr'].nunique()
encounter_number, patient_number

(38314, 31395)

We have data from 31395 diabetic patients, covering 38314 different diabetic encounters.

**First finding**: Although all of these encounters are diabetic visit, circulatory diseases like heart diseases are very likely to happen at the same time.

**Second finding**: Only a small amount patients have further encounters so we can expect that this dataset should be an imbalanced dataset when we build the classification model.

### Collapsing of Multiple Encounters for same patient
For some patients in the dataset had more than one encounter, we could not conside them as independent encounters because that bias the results towards those patients who had multiple encounters. Thus we decided to use first encounters of patients with multiple encounters.


In [11]:
df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
df.shape

(31395, 50)

## Data cleaning
After this first look, we should perform data cleanning and feature engineering, before doing any further analysis.

### Dealing with missing values
First we have to see how many missing values are (which were coded as “?” or "None" in the data):

In [12]:
for col in df.columns:
    if df[col].dtype == object:
        print(col,df[col][(df[col] == '?')|(df[col] == 'None')].count()/ len(df))

race 0.027201783723522853
gender 0.0
age 0.0
weight 0.9748049052396879
payer_code 0.4576206402293359
medical_specialty 0.44994425863991083
max_glu_serum 0.9446408663799968
A1Cresult 0.7823220258002866
metformin 0.0
repaglinide 0.0
nateglinide 0.0
chlorpropamide 0.0
glimepiride 0.0
acetohexamide 0.0
glipizide 0.0
glyburide 0.0
tolbutamide 0.0
pioglitazone 0.0
rosiglitazone 0.0
acarbose 0.0
miglitol 0.0
troglitazone 0.0
tolazamide 0.0
examide 0.0
citoglipton 0.0
insulin 0.0
glyburide-metformin 0.0
glipizide-metformin 0.0
glimepiride-pioglitazone 0.0
metformin-rosiglitazone 0.0
metformin-pioglitazone 0.0
change 0.0
diabetesMed 0.0
readmitted 0.0
c_diag1 0.00012740882306099699
c_diag2 0.007071189679885332
c_diag3 0.03532409619366141


1. Weight and max_glu_serumis are missing in over 90% records. Owing to the poor interpretability of missing values and little predictive generalizability to other patients, we drop these two features.
2. Payer code and Medical Specialty of treating physician also have 40–50% missing values. Also these two features might have less relationship to our prediction result. We decided to drop these.
3. admission_type_id and admission_source_id are similar features and there are too many catelogs in admission_source_id, we decided to drop admission_source_id.

In [13]:
df = df.drop(['weight','max_glu_serum','payer_code','medical_specialty', 'admission_source_id'], axis = 1)
df.shape

(31395, 45)

Also, one more cleaning step that depends on understanding the data and some common sense: since we are trying to predict readmissions, those patients who died during this hospital admission, have zero probability of readmission, the same for discharge to hospice. So we should remove those records (discharge_disposition = 11, 13 or 14, 19, 20,21).

In [14]:
df = df[(df['discharge_disposition_id'] != int(11)) & (df['discharge_disposition_id'] != int(13)) & (df['discharge_disposition_id'] != int(14)) & (df['discharge_disposition_id'] != int(19)) & (df['discharge_disposition_id'] != int(20)) & (df['discharge_disposition_id'] != int(21))]

In [15]:
df.shape

(31114, 45)

### Creating and/or Recoding New Features
This is highly subjective, and partly depends on a knowledge of healthcare services, and making sense of the potential relationships between features. We tried some (none are perfect).

**Service utilization**: The data contains variables for number of inpatient (admissions), emergency room visits and outpatient visits for a given patient in the previous one year. These are (crude) measures of how much hospital/clinic services a person has used in the past year. We added these three to create a new variable called service utilization. The idea was to see which version gives us better results. Granted, we did not apply any special weighting to the three ingredients of service utilization but we wanted to try something simple at this stage.

In [16]:
df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

In [17]:
df.shape

(31114, 46)

In [18]:
df = df.drop(['number_outpatient','number_emergency','number_inpatient'], axis = 1)
df.shape

(31114, 43)

To check the result of doing this, we use the value_counts() method which gives a nice tapering distribution:

In [19]:
df['service_utilization'].value_counts()

0     21588
1      4877
2      2102
3      1096
4       556
5       321
6       205
7       117
8        64
9        47
10       32
11       30
12       20
13       13
14       12
15       11
16        7
17        3
26        2
20        2
21        1
25        1
27        1
19        1
30        1
18        1
23        1
49        1
24        1
Name: service_utilization, dtype: int64

**Number of medication changes**: The dataset contains 23 features for 23 drugs (or combos) which indicate for each of these, whether a change in that medication was made or not during the current hospital stay of patient. Medication change for diabetics upon admission has been shown by previous research to be associated with lower readmission rates. We decided to count how many changes were made in total for each patient, and declared that a new feature. The reasoning here was to both simplify the model and possibly discover a relationship with number of changes regardless of which drug was changed. In python this is done by:

In [20]:
keys = ['citoglipton', 'examide', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']
for col in keys:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
df['numMedchange'] = 0
for col in keys:
    colname = str(col) + 'temp'
    df['numMedchange'] = df['numMedchange'] + df[colname]
    del df[colname]

In [21]:
df.shape

(31114, 44)

In [22]:
df = df.drop(['citoglipton', 'examide', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide'], axis = 1)
df.shape

(31114, 21)

To check the result of doing this, we use the value_counts() method which gives a nice tapering distribution:

In [23]:
df['numMedchange'].value_counts()

0    22142
1     8454
2      466
3       49
4        3
Name: numMedchange, dtype: int64

## Reformatting some other variables
Just like diagnoses, there were quite a few categories for admission source, admission type and discharge disposition. We collapsed these variables into fewer categories where it made sense. We also transform some text catelogical variables into numerical variables.

Admission_type:
1. Emergency
3. Elective
4. Newborn
5. NULL/not mapped

In [24]:
df['admission_type_id'] = df['admission_type_id'].replace(2,1)
df['admission_type_id'] = df['admission_type_id'].replace(7,1)
df['admission_type_id'] = df['admission_type_id'].replace(6,5)
df['admission_type_id'] = df['admission_type_id'].replace(8,5)
df['admission_type_id'].value_counts()

1    21204
3     6300
5     3606
4        4
Name: admission_type_id, dtype: int64

Discharge_disposition:
1. Go to home
2. Tranform to other health care
3. Left AMA
4. Newborn tranform to other health care
5. NULL/not mapped

In [25]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(3,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(4,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(5,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(6,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(8,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(9,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(12,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(15,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(16,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(17,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(22,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(23,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(24,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(30,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(27,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(28,2)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(25,18)

In [26]:
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(7,3)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(10,4)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(18,5)
df['discharge_disposition_id'].value_counts()

1    21136
2     8057
5     1695
3      221
4        5
Name: discharge_disposition_id, dtype: int64

For the catalog of newborn is quite small both in admission and discharge, we remove those records (discharge_disposition_id = 4 or admission_type_id = 4).

In [27]:
df = df[(df['discharge_disposition_id'] == int(4)) | (df['admission_type_id'] != int(4))]
df.shape

(31110, 21)

In [28]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)
df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)
df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

In [29]:
df['A1Cresult'] = df['A1Cresult'].replace('>7', 1)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 1)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 0)
df['A1Cresult'] = df['A1Cresult'].replace('None', -99)

In [30]:
df['age'] = df['age'].replace('[0-10)',5)
df['age'] = df['age'].replace('[10-20)',15)
df['age'] = df['age'].replace('[20-30)',25)
df['age'] = df['age'].replace('[30-40)',35)
df['age'] = df['age'].replace('[40-50)',45)
df['age'] = df['age'].replace('[50-60)',55)
df['age'] = df['age'].replace('[60-70)',65)
df['age'] = df['age'].replace('[70-80)',75)
df['age'] = df['age'].replace('[80-90)',85)
df['age'] = df['age'].replace('[90-100)',95)

**Reforming the outcome variable**: The outcome we are looking at is whether the patient gets readmitted to the hospital. The variable actually has < 30, > 30 and No Readmission categories. To reduce our problem to a binary classification, we combined the readmission after 30 days and no readmission into a single category.

In [31]:
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [32]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                       int64
age                          int64
admission_type_id            int64
discharge_disposition_id     int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_diagnoses             int64
A1Cresult                    int64
change                       int64
diabetesMed                  int64
readmitted                   int64
c_diag1                     object
c_diag2                     object
c_diag3                     object
service_utilization          int64
numMedchange                 int64
dtype: object

In [33]:
df.to_csv('./diabetes_data_processed.csv')